In [1]:
import cv2
import numpy as np
import getopt
import sys
import random

In [2]:
def findFeatures(img):
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    
    return keypoints, descriptors

In [3]:
def matchFeatures(kp1, kp2, desc1, desc2, img1, img2):
    matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = matcher.match(desc1, desc2)

    return matches

In [4]:
#
# Computers a homography from 4-correspondences
#
def calculateHomography(correspondences):
    #loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),
              p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
        a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,
              p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
        aList.append(a1)
        aList.append(a2)
 
    matrixA = np.matrix(aList)

    #svd composition
    u, s, v = np.linalg.svd(matrixA)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    
    return h

In [5]:
img1 = cv2.imread('img1.png', 0)
img2 = cv2.imread('img4.png', 0)

In [6]:
kp1, desc1 = findFeatures(img1)
kp2, desc2 = findFeatures(img2)

[ WARN:0@1.536] global /io/opencv_contrib/modules/xfeatures2d/misc/python/shadow_sift.hpp (13) SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


In [17]:
#find features and keypoints
correspondenceList = []

keypoints = [kp1, kp2]
matches = matchFeatures(kp1, kp2, desc1, desc2, img1, img2)

for match in matches:
    (x1, y1) = keypoints[0][match.queryIdx].pt
    (x2, y2) = keypoints[1][match.trainIdx].pt
    correspondenceList.append([x1, y1, x2, y2])

corrs = np.matrix(correspondenceList)

In [19]:
print(corrs.shape)

print(corrs)

(531, 4)
[[ 52.77297974 370.13537598 319.41482544 210.87768555]
 [128.76858521 223.2722168  298.61282349   3.69074297]
 [141.84231567 239.23823547 359.66296387 205.27929688]
 ...
 [667.41192627 264.00112915 122.38134003 211.09960938]
 [734.58782959 172.56782532 274.09265137 210.59173584]
 [796.47229004 539.5637207  216.98129272 187.09721375]]


In [20]:
p1 = np.matrix([111, 222, 1])

In [23]:
print(p1)

[[111 222   1]]
